In [1]:
import climtas
import xarray
from glob import glob

In [2]:
import os
import dask.distributed

# Edit as desired
threads_per_worker = 1

try:
    c # Already running
except NameError:
    c = dask.distributed.Client(
        n_workers=int(os.environ.get('PBS_NCPUS', 1))//threads_per_worker,
        threads_per_worker=threads_per_worker,
        memory_limit=f'{2*threads_per_worker}gb',
        local_directory=os.path.join(os.environ.get('PBS_JOBFS', os.environ['TMPDIR']),
                                     'dask-worker-space')
    )
c

Client Scheduler: tcp://127.0.0.1:45863 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 2.00 GB


In [3]:
# Open the dataset

ds = xarray.open_mfdataset(sorted(glob('/g/data/ub4/era5/netcdf/surface/2T/201[56]/2T_era5_global_*.nc')),
                           combine='nested',
                           concat_dim='time',
                           chunks={'latitude': 50, 'longitude': 50})
ds

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 17544)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2016-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(744, 50, 50), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2018-10-28 06:45:46 GMT by grib_to_netcdf-2.7.0: /opt/ecmwf...

In [4]:
# Convert to daily mean - or max, min etc.

t2m_daily = ds.t2m.resample(time='D').mean()
t2m_daily

<xarray.DataArray 't2m' (time: 731, latitude: 721, longitude: 1440)>
dask.array<stack, shape=(731, 721, 1440), dtype=float32, chunksize=(1, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2016-12-31
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0

In [5]:
# Smooth out the data - the daily values have chunksize = 1 in the time 
# dimension, we need to extend that for the rolling operation

t2m_smooth = t2m_daily.chunk({'time': 20}).rolling(time=15, center=True).mean()
t2m_smooth

<xarray.DataArray (time: 731, latitude: 721, longitude: 1440)>
dask.array<where, shape=(731, 721, 1440), dtype=float32, chunksize=(20, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2016-12-31
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0

In [6]:
# Calculate percentiles
# The rolling mean will leave some NAN values at the start and end of the
# time series, these must be removed (e.g. by doing the rolling on a larger
# timeseries than you need and trimming the NANs off before the percentile)

t2m_percentile = climtas.apply_doy.percentile_doy(t2m_smooth, 90)
t2m_percentile

<xarray.DataArray (latitude: 721, longitude: 1440, dayofyear: 366)>
dask.array<group_func, shape=(721, 1440, 366), dtype=float32, chunksize=(50, 50, 366), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366

In [7]:
# Use the throttled saver to write to netcdf one chunk at a time

climtas.io.to_netcdf_throttled(t2m_percentile.to_dataset(name='t2m_percentile'),
                               '/g/data/w35/saw562/era5_heatwave_clim.nc')

RuntimeError: NetCDF: HDF error